# AIS Vessel Trajectory Prediction - Visual Analysis

This notebook provides a complete visual walkthrough of our vessel H3 cell prediction system:

1. **Raw Vessel Data Exploration** - See what data we have
2. **H3 Grid Visualization** - Understand the spatial framework 
3. **Feature Engineering** - Transform raw data into ML features
4. **Model Training & Results** - Train and evaluate our predictor
5. **Prediction Visualization** - See model predictions on maps

Let's start exploring! 🚢🗺️

In [ ]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import h3
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add our source code to path
sys.path.append('../src')

from features.vessel_h3_tracker import VesselH3Tracker
from features.vessel_features import VesselFeatureExtractor

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("📦 All libraries imported successfully!")
print("🚀 Ready to explore vessel trajectory prediction!")

## 1. Raw Vessel Data Exploration 🛳️

Let's start by loading and exploring our raw AIS vessel data to understand what we're working with.

In [ ]:
# Load raw vessel data
print("📊 Loading raw vessel data...")
df = pd.read_pickle('../data/raw/ais_cape_data_2024.pkl')

print(f"✅ Loaded {len(df):,} AIS records")
print(f"📅 Date range: {df['mdt'].min()} to {df['mdt'].max()}")
print(f"🚢 Unique vessels: {df['imo'].nunique()}")
print(f"📍 Geographic bounds:")
print(f"   Latitude: {df['lat'].min():.3f} to {df['lat'].max():.3f}")
print(f"   Longitude: {df['lon'].min():.3f} to {df['lon'].max():.3f}")

# Show sample data
print("\n📋 Sample data:")
display(df[['imo', 'mdt', 'lat', 'lon', 'speed', 'heading', 'destination']].head())

In [ ]:
# Analyze vessel activity
vessel_counts = df['imo'].value_counts()

# Plot vessel activity distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Records per vessel
axes[0,0].hist(vessel_counts.values, bins=50, alpha=0.7, color='skyblue')
axes[0,0].set_title('Distribution of Records per Vessel')
axes[0,0].set_xlabel('Number of AIS Records')
axes[0,0].set_ylabel('Number of Vessels')
axes[0,0].axvline(vessel_counts.median(), color='red', linestyle='--', label=f'Median: {vessel_counts.median():.0f}')
axes[0,0].legend()

# 2. Speed distribution
axes[0,1].hist(df['speed'].dropna(), bins=50, alpha=0.7, color='lightgreen')
axes[0,1].set_title('Vessel Speed Distribution')
axes[0,1].set_xlabel('Speed (knots)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].axvline(df['speed'].median(), color='red', linestyle='--', label=f'Median: {df["speed"].median():.1f} knots')
axes[0,1].legend()

# 3. Geographic distribution
axes[1,0].scatter(df['lon'], df['lat'], alpha=0.1, s=0.5, color='navy')
axes[1,0].set_title('Geographic Distribution of AIS Points')
axes[1,0].set_xlabel('Longitude')
axes[1,0].set_ylabel('Latitude')
axes[1,0].grid(True, alpha=0.3)

# 4. Temporal distribution
df['hour'] = df['mdt'].dt.hour
hourly_counts = df['hour'].value_counts().sort_index()
axes[1,1].bar(hourly_counts.index, hourly_counts.values, alpha=0.7, color='orange')
axes[1,1].set_title('AIS Messages by Hour of Day')
axes[1,1].set_xlabel('Hour of Day')
axes[1,1].set_ylabel('Number of Messages')

plt.tight_layout()
plt.show()

print(f"📊 Top 5 most active vessels:")
for i, (imo, count) in enumerate(vessel_counts.head().items()):
    print(f"   {i+1}. Vessel {imo}: {count:,} records")

## 2. H3 Grid Visualization 🗺️

Now let's visualize our H3 hexagonal grid system and see how it overlays with the vessel data.

In [ ]:
# Select our test vessel (most active one)
test_vessel_imo = vessel_counts.index[0]
test_vessel_data = df[df['imo'] == test_vessel_imo].head(200).copy()  # First 200 points for visualization

print(f"🚢 Selected vessel {test_vessel_imo} for detailed analysis")
print(f"📊 Using {len(test_vessel_data)} data points")
print(f"📅 Time span: {test_vessel_data['mdt'].min()} to {test_vessel_data['mdt'].max()}")

# Convert to H3 cells
tracker = VesselH3Tracker(h3_resolution=5)
h3_sequence = tracker.convert_vessel_to_h3_sequence(test_vessel_data)

print(f"✅ Converted to {len(h3_sequence)} H3 positions")
print(f"🗺️ H3 resolution 5: ~{h3.edge_length(5, unit='km'):.1f}km cell edge length")

# Get unique H3 cells visited
unique_cells = h3_sequence['h3_cell'].unique()
print(f"📍 Vessel visited {len(unique_cells)} unique H3 cells")

display(h3_sequence[['mdt', 'lat', 'lon', 'h3_cell', 'speed', 'heading']].head())

In [ ]:
# Create an interactive map with H3 grid overlay
def create_h3_vessel_map(vessel_data, h3_sequence):
    """Create an interactive map showing vessel track with H3 grid"""
    
    # Calculate map center
    center_lat = vessel_data['lat'].mean()
    center_lon = vessel_data['lon'].mean()
    
    # Create base map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=10,
        tiles='OpenStreetMap'
    )
    
    # Add vessel track as a line
    track_coords = [[row['lat'], row['lon']] for _, row in vessel_data.iterrows()]
    folium.PolyLine(
        track_coords,
        color='blue',
        weight=3,
        opacity=0.8,
        popup='Vessel Track'
    ).add_to(m)
    
    # Add H3 cells as hexagons
    unique_cells = h3_sequence['h3_cell'].unique()
    
    for i, cell_id in enumerate(unique_cells[:50]):  # Limit to first 50 for performance
        # Get hexagon boundary
        boundary = h3.h3_to_geo_boundary(cell_id, geo_json=True)
        
        # Color based on visit order
        color = plt.cm.viridis(i / min(len(unique_cells), 50))
        hex_color = f"#{int(color[0]*255):02x}{int(color[1]*255):02x}{int(color[2]*255):02x}"
        
        folium.Polygon(
            boundary,
            color=hex_color,
            weight=2,
            fillColor=hex_color,
            fillOpacity=0.3,
            popup=f'H3 Cell: {cell_id}'
        ).add_to(m)
    
    # Add start and end markers
    start_point = vessel_data.iloc[0]
    end_point = vessel_data.iloc[-1]
    
    folium.Marker(
        [start_point['lat'], start_point['lon']],
        popup='Start',
        icon=folium.Icon(color='green', icon='play')
    ).add_to(m)
    
    folium.Marker(
        [end_point['lat'], end_point['lon']],
        popup='End',
        icon=folium.Icon(color='red', icon='stop')
    ).add_to(m)
    
    return m

# Create the map
print("🗺️ Creating interactive H3 grid map...")
vessel_map = create_h3_vessel_map(test_vessel_data, h3_sequence)

# Save and display
map_path = '../visualizations/vessel_h3_training_data.html'
vessel_map.save(map_path)
print(f"💾 Map saved to: {map_path}")

# Display in notebook
vessel_map

## 3. Feature Engineering Transformation 🔧

Now let's see how we transform the raw vessel positions into the 65 features used for machine learning.

In [ ]:
# Extract features using our feature engineering pipeline
print("🔧 Extracting vessel features...")
extractor = VesselFeatureExtractor(h3_resolution=5)
features_df = extractor.extract_all_features(h3_sequence)

print(f"✅ Extracted {len(features_df.columns)} features for {len(features_df)} time points")
print(f"📊 Feature categories include:")
print("   - Core state (position, speed, heading)")
print("   - Historical patterns (6h, 12h, 24h windows)")
print("   - Movement characteristics (efficiency, trends)")
print("   - Contextual information (journey phase, cargo status)")

# Show sample features
key_features = ['current_h3_cell', 'current_speed', 'current_heading', 'time_in_current_cell', 
                'avg_speed_6h', 'speed_trend_6h', 'journey_phase', 'likely_cargo_status']

print("\n📋 Sample engineered features:")
display(features_df[key_features].head(10))

In [ ]:
# Visualize feature distributions and patterns
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Speed over time
axes[0,0].plot(features_df.index, features_df['current_speed'], alpha=0.7, label='Current Speed')
axes[0,0].plot(features_df.index, features_df['avg_speed_6h'], alpha=0.7, label='6h Average')
axes[0,0].set_title('Speed Patterns')
axes[0,0].set_xlabel('Time Index')
axes[0,0].set_ylabel('Speed (knots)')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Time in cell distribution
axes[0,1].hist(features_df['time_in_current_cell'].dropna(), bins=20, alpha=0.7, color='lightblue')
axes[0,1].set_title('Time Spent in H3 Cells')
axes[0,1].set_xlabel('Time in Cell')
axes[0,1].set_ylabel('Frequency')

# 3. Movement efficiency
axes[0,2].hist(features_df['movement_efficiency_6h'].dropna(), bins=20, alpha=0.7, color='lightgreen')
axes[0,2].set_title('Movement Efficiency (6h)')
axes[0,2].set_xlabel('Efficiency Ratio')
axes[0,2].set_ylabel('Frequency')

# 4. Cells visited over time
axes[1,0].plot(features_df.index, features_df['cells_visited_6h'], alpha=0.7, color='orange')
axes[1,0].set_title('H3 Cells Visited (6h window)')
axes[1,0].set_xlabel('Time Index')
axes[1,0].set_ylabel('Number of Cells')
axes[1,0].grid(True, alpha=0.3)

# 5. Journey phase distribution
phase_counts = features_df['journey_phase'].value_counts()
axes[1,1].pie(phase_counts.values, labels=phase_counts.index, autopct='%1.1f%%')
axes[1,1].set_title('Journey Phase Distribution')

# 6. Speed trend patterns
trend_counts = features_df['speed_trend_6h'].value_counts()
axes[1,2].bar(trend_counts.index, trend_counts.values, alpha=0.7, color='purple')
axes[1,2].set_title('Speed Trend Patterns (6h)')
axes[1,2].set_xlabel('Trend')
axes[1,2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print("📊 Feature engineering summary:")
print(f"   - Non-null features: {features_df.count().sum()} / {len(features_df) * len(features_df.columns)}")
print(f"   - Completeness: {(features_df.count().sum() / (len(features_df) * len(features_df.columns)) * 100):.1f}%")

## 4. Training Data Creation 📊

Let's create the training sequences and see what our model will learn from.

In [ ]:
# Create training sequences (input → target pairs)
print("📊 Creating training sequences...")

sequences = []
for i in range(len(features_df) - 1):  # -1 because we need next cell
    current_row = features_df.iloc[i]
    next_row = features_df.iloc[i + 1]
    
    # Simple input features
    input_features = {
        'current_h3_cell': current_row['current_h3_cell'],
        'current_speed': current_row['current_speed'],
        'current_heading': current_row['current_heading'],
        'lat': current_row['lat'],
        'lon': current_row['lon'],
        'time_in_current_cell': current_row['time_in_current_cell']
    }
    
    # Target: next H3 cell
    target = next_row['current_h3_cell']
    
    # Combine
    sequence = {**input_features, 'target_h3_cell': target}
    sequences.append(sequence)

training_df = pd.DataFrame(sequences)

print(f"✅ Created {len(training_df)} training sequences")
print(f"🎯 Input features: {list(training_df.columns[:-1])}")
print(f"🎯 Target: {training_df.columns[-1]}")

# Analyze the training data
unique_current = training_df['current_h3_cell'].nunique()
unique_targets = training_df['target_h3_cell'].nunique()

print(f"\n📊 Training data analysis:")
print(f"   - Unique current cells: {unique_current}")
print(f"   - Unique target cells: {unique_targets}")
print(f"   - Average speed: {training_df['current_speed'].mean():.1f} knots")
print(f"   - Speed range: {training_df['current_speed'].min():.1f} - {training_df['current_speed'].max():.1f} knots")

display(training_df.head())

In [ ]:
# Visualize training patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Cell transition matrix (sample)
transition_counts = training_df.groupby(['current_h3_cell', 'target_h3_cell']).size().reset_index(name='count')
top_transitions = transition_counts.nlargest(10, 'count')

axes[0,0].barh(range(len(top_transitions)), top_transitions['count'])
axes[0,0].set_title('Top 10 H3 Cell Transitions')
axes[0,0].set_xlabel('Frequency')
axes[0,0].set_ylabel('Transition Rank')

# 2. Speed vs heading relationship
scatter = axes[0,1].scatter(training_df['current_heading'], training_df['current_speed'], 
                          alpha=0.5, s=10, c=training_df.index, cmap='viridis')
axes[0,1].set_title('Speed vs Heading Pattern')
axes[0,1].set_xlabel('Heading (degrees)')
axes[0,1].set_ylabel('Speed (knots)')
plt.colorbar(scatter, ax=axes[0,1], label='Time sequence')

# 3. Geographic distribution of training points
axes[1,0].scatter(training_df['lon'], training_df['lat'], alpha=0.5, s=10, color='navy')
axes[1,0].set_title('Geographic Distribution of Training Data')
axes[1,0].set_xlabel('Longitude')
axes[1,0].set_ylabel('Latitude')
axes[1,0].grid(True, alpha=0.3)

# 4. Cell visit frequency
cell_counts = training_df['current_h3_cell'].value_counts()
axes[1,1].hist(cell_counts.values, bins=20, alpha=0.7, color='orange')
axes[1,1].set_title('H3 Cell Visit Frequency')
axes[1,1].set_xlabel('Number of Visits')
axes[1,1].set_ylabel('Number of Cells')

plt.tight_layout()
plt.show()

print(f"📊 Most visited cells:")
for i, (cell, count) in enumerate(cell_counts.head().items()):
    print(f"   {i+1}. {cell}: {count} visits")

## 5. Model Training & Evaluation 🤖

Now let's train our Random Forest model and see how well it performs.

In [ ]:
# Load our trained model (if it exists) or train a new one
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import joblib

# Check if we have a trained model
model_path = '../data/models/final_models/simple_h3_predictor.pkl'
encoder_path = '../data/models/final_models/h3_label_encoder.pkl'

if os.path.exists(model_path) and os.path.exists(encoder_path):
    print("📦 Loading existing trained model...")
    model = joblib.load(model_path)
    h3_encoder = joblib.load(encoder_path)
    print("✅ Model loaded successfully!")
else:
    print("🤖 Training new model...")
    
    # Prepare data
    h3_encoder = LabelEncoder()
    all_h3_cells = list(set(training_df['current_h3_cell'].tolist() + training_df['target_h3_cell'].tolist()))
    h3_encoder.fit(all_h3_cells)
    
    # Features (X) and target (y)
    X = training_df[['current_speed', 'current_heading', 'lat', 'lon', 'time_in_current_cell']].copy()
    X['current_h3_encoded'] = h3_encoder.transform(training_df['current_h3_cell'])
    y = h3_encoder.transform(training_df['target_h3_cell'])
    
    # Train model
    model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=15)
    model.fit(X, y)
    
    # Save model
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    joblib.dump(model, model_path)
    joblib.dump(h3_encoder, encoder_path)
    print("💾 Model saved!")

print(f"🌲 Model: RandomForest with {model.n_estimators} trees")
print(f"🗺️ Predicting among {len(h3_encoder.classes_)} possible H3 cells")

In [ ]:
# Evaluate model performance
print("📊 Evaluating model performance...")

# Prepare test data
X = training_df[['current_speed', 'current_heading', 'lat', 'lon', 'time_in_current_cell']].copy()
X['current_h3_encoded'] = h3_encoder.transform(training_df['current_h3_cell'])
y = h3_encoder.transform(training_df['target_h3_cell'])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Make predictions
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = accuracy_score(y_test, test_pred)

print(f"📈 Training Accuracy: {train_accuracy:.1%}")
print(f"📈 Test Accuracy: {test_accuracy:.1%}")

# Feature importance
feature_names = X.columns
importances = model.feature_importances_
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)

print("\n🔍 Feature Importance:")
for _, row in feature_importance.iterrows():
    print(f"   {row['feature']}: {row['importance']:.3f}")

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.title('Feature Importance in H3 Cell Prediction')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

## 6. Prediction Visualization 🔮

Let's see our model's predictions in action on a map!

In [ ]:
# Make predictions for a sequence
print("🔮 Making predictions for test sequence...")

# Take a subset for prediction visualization
test_sequence = training_df.iloc[50:80].copy()  # 30 points for visualization

predictions = []
for _, row in test_sequence.iterrows():
    # Prepare input
    X_input = [[row['current_speed'], row['current_heading'], row['lat'], 
                row['lon'], row['time_in_current_cell'], 
                h3_encoder.transform([row['current_h3_cell']])[0]]]
    
    # Predict
    pred_encoded = model.predict(X_input)[0]
    pred_h3 = h3_encoder.inverse_transform([pred_encoded])[0]
    
    predictions.append({
        'current_h3': row['current_h3_cell'],
        'actual_next_h3': row['target_h3_cell'],
        'predicted_next_h3': pred_h3,
        'correct': pred_h3 == row['target_h3_cell'],
        'lat': row['lat'],
        'lon': row['lon'],
        'speed': row['current_speed'],
        'heading': row['current_heading']
    })

pred_df = pd.DataFrame(predictions)
accuracy_subset = pred_df['correct'].mean()

print(f"✅ Made predictions for {len(pred_df)} points")
print(f"📊 Accuracy on this subset: {accuracy_subset:.1%}")
print(f"🎯 Correct predictions: {pred_df['correct'].sum()}/{len(pred_df)}")

display(pred_df[['current_h3', 'actual_next_h3', 'predicted_next_h3', 'correct', 'speed']].head(10))

In [ ]:
# Create prediction visualization map
def create_prediction_map(pred_df):
    """Create map showing predictions vs actual movements"""
    
    center_lat = pred_df['lat'].mean()
    center_lon = pred_df['lon'].mean()
    
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=12,
        tiles='OpenStreetMap'
    )
    
    # Add prediction points
    for i, row in pred_df.iterrows():
        # Color based on correctness
        color = 'green' if row['correct'] else 'red'
        icon = 'ok' if row['correct'] else 'remove'
        
        # Current position marker
        folium.Marker(
            [row['lat'], row['lon']],
            popup=f"""Point {i+1}<br>
                     Speed: {row['speed']:.1f} knots<br>
                     Current: {row['current_h3'][-6:]}<br>
                     Actual: {row['actual_next_h3'][-6:]}<br>
                     Predicted: {row['predicted_next_h3'][-6:]}<br>
                     Correct: {row['correct']}""",
            icon=folium.Icon(color=color, icon=icon)
        ).add_to(m)
        
        # Add actual H3 cell boundary
        actual_boundary = h3.h3_to_geo_boundary(row['actual_next_h3'], geo_json=True)
        folium.Polygon(
            actual_boundary,
            color='blue',
            weight=2,
            fillColor='blue',
            fillOpacity=0.2,
            popup=f'Actual: {row["actual_next_h3"][-6:]}'
        ).add_to(m)
        
        # Add predicted H3 cell boundary (if different)
        if row['predicted_next_h3'] != row['actual_next_h3']:
            pred_boundary = h3.h3_to_geo_boundary(row['predicted_next_h3'], geo_json=True)
            folium.Polygon(
                pred_boundary,
                color='red',
                weight=2,
                fillColor='red',
                fillOpacity=0.2,
                popup=f'Predicted: {row["predicted_next_h3"][-6:]}'
            ).add_to(m)
    
    # Add legend
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 90px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px">
    <b>Prediction Legend</b><br>
    🟢 Correct Prediction<br>
    🔴 Incorrect Prediction<br>
    🟦 Actual Next Cell<br>
    🟥 Predicted Next Cell
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    return m

print("🗺️ Creating prediction visualization map...")
prediction_map = create_prediction_map(pred_df)

# Save map
pred_map_path = '../visualizations/vessel_predictions_map.html'
prediction_map.save(pred_map_path)
print(f"💾 Prediction map saved to: {pred_map_path}")

# Display map
prediction_map

## 7. Results Summary 📊

Let's summarize what we've accomplished and learned from this analysis.

In [ ]:
# Generate comprehensive summary
print("=" * 60)
print("🚢 VESSEL TRAJECTORY PREDICTION - RESULTS SUMMARY")
print("=" * 60)

print(f"\n📊 DATA PIPELINE:")
print(f"   Raw AIS Records: {len(df):,}")
print(f"   Vessels Available: {df['imo'].nunique()}")
print(f"   Test Vessel: {test_vessel_imo}")
print(f"   Analysis Points: {len(test_vessel_data)}")

print(f"\n🗺️ SPATIAL FRAMEWORK:")
print(f"   H3 Resolution: 5")
print(f"   Cell Edge Length: ~{h3.edge_length(5, unit='km'):.1f} km")
print(f"   Unique Cells Visited: {len(unique_cells)}")
print(f"   H3 Sequence Length: {len(h3_sequence)}")

print(f"\n🔧 FEATURE ENGINEERING:")
print(f"   Total Features: {len(features_df.columns)}")
print(f"   Feature Categories: Core state, Historical patterns, Movement characteristics, Context")
print(f"   Data Completeness: {(features_df.count().sum() / (len(features_df) * len(features_df.columns)) * 100):.1f}%")

print(f"\n🤖 MACHINE LEARNING:")
print(f"   Model: Random Forest")
print(f"   Training Samples: {len(training_df)}")
print(f"   Prediction Classes: {len(h3_encoder.classes_)} H3 cells")
print(f"   Test Accuracy: {test_accuracy:.1%}")

print(f"\n🎯 KEY INSIGHTS:")
print(f"   Most Important Features: {', '.join(feature_importance.head(3)['feature'].values)}")
print(f"   Geographic Factors: Location (lat/lon) are highly predictive")
print(f"   Movement Factors: Current H3 cell and speed matter significantly")
print(f"   Prediction Challenge: {len(h3_encoder.classes_)} possible destinations make this complex")

print(f"\n🚀 NEXT STEPS FOR IMPROVEMENT:")
print(f"   1. Add more vessels to training data")
print(f"   2. Use sequential models (LSTM) instead of Random Forest")
print(f"   3. Include more of the 65 available features")
print(f"   4. Implement temporal sequence prediction")
print(f"   5. Add external factors (weather, traffic)")

print("\n" + "=" * 60)
print("✅ ANALYSIS COMPLETE - WORKING ML PIPELINE ESTABLISHED!")
print("=" * 60)

In [ ]:
# Save analysis results
results_summary = {
    'dataset_size': len(df),
    'test_vessel': str(test_vessel_imo),
    'h3_resolution': 5,
    'unique_cells': len(unique_cells),
    'feature_count': len(features_df.columns),
    'training_samples': len(training_df),
    'test_accuracy': float(test_accuracy),
    'model_type': 'RandomForest',
    'feature_importance': feature_importance.to_dict('records')
}

import json
results_path = '../data/processed/analysis_results.json'
with open(results_path, 'w') as f:
    json.dump(results_summary, f, indent=2)

print(f"💾 Analysis results saved to: {results_path}")
print("🎉 Notebook analysis complete!")
print("\n📁 Generated files:")
print(f"   - Vessel track map: ../visualizations/vessel_h3_training_data.html")
print(f"   - Prediction map: ../visualizations/vessel_predictions_map.html")
print(f"   - Analysis results: {results_path}")